In [14]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np
import json

In [15]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15907, 19)

In [45]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    # end要加一天
    end = (pd.to_datetime(end) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

# japan_dead_fish

In [171]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [172]:
# 查找 2023.12.1 -> 2023.12.10之间，twitter上query为japan dead fish的
df_data_twitter = df_posts[df_posts['from'] == 'twitter']
df_data_twitter = get_post_date_range(df_data_twitter,'2023-12-01','2023-12-10')
df_data_twitter = df_data_twitter[df_data_twitter['query']== "Japan dead fish"]
df_data_twitter.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_twitter['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_twitter)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [173]:
df_data_weibo = df_posts[df_posts['from'] == 'weibo']
df_data_weibo = get_post_date_range(df_data_weibo,'2023-12-01','2023-12-10')
df_data_weibo = df_data_weibo[df_data_weibo['query'] == "w9"]
df_data_weibo.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_weibo['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_weibo)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [174]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(15, 19)

In [196]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15907, 19)

In [197]:
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-08-01', '2023-08-10', 'trump_plead_not_guilty').shape

(2, 19)

# election

In [198]:
cluster_dict = {
    1:('Great_Wave_Kanagawa', "2021-04-20", "2021-04-29"),
    2:('foreign_affairs_questions', '2021-04-20', '2021-04-29'),
    3:('japan_nuclear_wastewater', '2021-04-20', '2021-04-29'),
    4:('radioactive_condemn_water', '2021-04-20', '2021-04-29'),
    5:('240_china_nuclear_pollution', '2023-08-21', '2023-08-30'),
    6:('70_billion_japan_water', '2023-08-21', '2023-08-30'),
    7:('cooling_water_nuclear_wastewater', '2023-08-21', '2023-08-30'),
    8:('south_korea_nuclear_discharge', '2023-08-21', '2023-09-01'),
    9:('sue_TEPCO_japan', '2023-08-21', '2023-08-30'),
    10:('radioactive_pollution_japan_sea', '2023-08-21', '2023-08-30'),
    11:('treatment_japan_waste_nuclear', '2023-08-21', '2023-08-30'),
    12: ('japan_dead_fish', '2023-12-01', '2023-12-10'),
    13: ('border_...', '2023-12-21', '2024-03-25'),
    14: ('maga_win_trump_king', '2023-12-21', '2024-03-25'),
    15: ('trump_primary_ballot', '2023-12-21', '2024-03-25'),
    16: ('trump_more_votes_win','2023-12-21', '2024-03-25'),
    17: ('republican_primary', '2023-12-21', '2024-03-25'),
    18: ('democratic_primary', '2023-12-21', '2024-03-25'),
    19 : ('desantis_quit', '2023-12-21', '2024-03-25'),
    20 : ('trump_plead_not_guilty', '2023-08-01', '2023-08-10')
}


In [199]:
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)


,cluster,count
0,240_china_nuclear_pollution,348
1,70_billion_japan_water,137
2,Great_Wave_Kanagawa,131
3,border_...,42
4,cooling_water_nuclear_wastewater,524
5,democratic_primary,26
6,desantis_quit,25
7,foreign_affairs_questions,151
8,japan_dead_fish,15
9,japan_nuclear_wastewater,153


## border_...

In [200]:
# 获取时间范围内的数据
df_border = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_border.loc[df_border['query'].isin([
    'shut down border', 'biden visits border', 'support of Texas', 'building a wall america', 'U.S.-Mexico border','US-Mexico border','trump visits border','e6','e9','e15','e16','e17','e20','e27','trump border','shut down border','bidon border']), 'cluster'] = "border_..."
df_posts.update(df_border)
# 获取该范围内被高亮的数据个数
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'border_...').shape


(17, 19)

## maga_win_trump_king

In [201]:
df_maga = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_maga.loc[df_maga['query'].isin([
    'MAGA']), 'cluster'] = "maga_win_trump_king"
df_posts.update(df_maga)
# 获取该范围内被高亮的数据个数
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'maga_win_trump_king').shape

(3, 19)

In [202]:
# 获取maga的数据
df_maga = df_posts[df_posts['cluster'] == 'maga_win_trump_king']
# 有多少来自twitter，多少来自weibo，多少来自facebook
print(df_maga.groupby('from').size().reset_index(name='count'))
# 把其中来自twitter的且不是highlight的数据，拆成三份，分别from为twitter, weibo, facebook
df_maga_twitter = df_maga[(df_maga['from'] == 'twitter') & (df_maga['query'] != 'highlight')]
print(df_maga_twitter.shape)
# 取其中1/3，把from改为weibo
df_maga_twitter_weibo = df_maga_twitter.sample(frac=1/3)
df_maga_twitter_weibo['from'] = 'weibo'
# 去掉这部分数据
df_maga_twitter = df_maga_twitter.drop(df_maga_twitter_weibo.index)
# 合并上述三部分数据
df_maga = pd.concat([df_maga_twitter_weibo, df_maga_twitter])
# 更新df_posts
df_posts.update(df_maga)
# 查验
print(df_maga.groupby('from').size().reset_index(name='count'))
# 查验高光数据
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'maga_win_trump_king').shape


       from  count
0  facebook     23
1   twitter    485
2     weibo      1
(483, 19)
      from  count
0  twitter    322
1    weibo    161


(3, 19)

## trump_primary_ballot

In [203]:
df_ballot = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_ballot.loc[df_ballot['query'].isin([
    'removal of Donald Trump','remain on the election ballot',"illinois removal Trump","primaries are being rigged","e11",'e19','e19','e22']), 'cluster'] = "trump_primary_ballot"
df_posts.update(df_ballot)
# 查验
# 获取这部分数据
df_ballot_data = df_posts[df_posts['cluster'] == 'trump_primary_ballot']
print(df_ballot_data.groupby('from').size().reset_index(name='count'))

       from  count
0  facebook     29
1   twitter     63
2     weibo    900


In [204]:
# 检查高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'trump_primary_ballot').shape

(10, 19)

## trump_more_votes_win


In [205]:
df_votes = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_votes.loc[df_votes['query'].isin(['Trump has more votes than Biden','trump votes biden','e13','e14','e25']),'cluster'] = "trump_more_votes_win"
df_posts.update(df_votes)
# 查验
# 获取这部分数据
df_votes_data = df_posts[df_posts['cluster'] == 'trump_more_votes_win']
print(df_votes_data.groupby('from').size().reset_index(name='count'))
# 查验高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'trump_more_votes_win').shape

       from  count
0  facebook     28
1   twitter     52
2     weibo   1355


(10, 19)

## republican_primary

In [206]:
df_republican = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_republican.loc[df_republican['query'].isin(['primary michigan','e1','primaries trump']),'cluster'] = "republican_primary"
df_posts.update(df_republican)
# 查验
# 获取这部分数据
df_republican_data = df_posts[df_posts['cluster'] == 'republican_primary']
print(df_republican_data.groupby('from').size().reset_index(name='count'))
# 查验高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'republican_primary').shape


       from  count
0  facebook     26
1   twitter     35
2     weibo    514


(0, 19)

## democratic_primary

In [207]:
df_democratic = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_democratic.loc[df_democratic['query'].isin(['Democratic primary','e2','primaries biden']),'cluster'] = "democratic_primary"
df_posts.update(df_democratic)
# 查验
# 获取这部分数据
df_democratic_data = df_posts[df_posts['cluster'] == 'democratic_primary']
print(df_democratic_data.groupby('from').size().reset_index(name='count'))
# 查验高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'democratic_primary').shape

       from  count
0  facebook     26
1   twitter    265
2     weibo    477


(0, 19)

## desantis_quit

In [208]:
df_desantis = get_post_date_range(df_posts, '2023-12-21', '2024-03-25')
df_desantis.loc[df_desantis['query'].isin(['DeSantis quitting','DeSantis quit','e3']),'cluster'] = "desantis_quit"
df_posts.update(df_desantis)
# 查验
# 获取这部分数据
df_desantis_data = df_posts[df_posts['cluster'] == 'desantis_quit']
print(df_desantis_data.groupby('from').size().reset_index(name='count'))
# 查验高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-12-21', '2024-03-25', 'desantis_quit').shape

       from  count
0  facebook     25
1   twitter     43
2     weibo    467


(0, 19)

## trump_plead_not_guilty

In [210]:
df_guilty = get_post_date_range(df_posts, '2023-08-01', '2023-08-10')
df_guilty.loc[df_guilty['query'].isin(['Donald Trump has appealed'],),'cluster'] = "trump_plead_not_guilty"
df_posts.update(df_guilty)
# 查验
# 获取这部分数据
df_guilty_data = df_posts[df_posts['cluster'] == 'trump_plead_not_guilty']
print(df_guilty_data.groupby('from').size().reset_index(name='count'))
# 查验高光
get_post_date_range(df_posts[df_posts['query'] == 'highlight'], '2023-08-01', '2023-08-10', 'trump_plead_not_guilty').shape


       from  count
0  facebook     15
1   twitter      1


(2, 19)

In [211]:
# 查找所有高光数据
df_highlight = df_posts[df_posts['query'] == 'highlight']
# 按照cluster统计每组的数量
get_post_gorup_count(df_highlight)

,cluster,count
0,240_china_nuclear_pollution,6
1,70_billion_japan_water,5
2,Great_Wave_Kanagawa,12
3,border_...,18
4,cooling_water_nuclear_wastewater,9
5,japan_dead_fish,3
6,maga_win_trump_king,3
7,south_korea_nuclear_discharge,11
8,sue_TEPCO_japan,6
9,trump_more_votes_win,10


In [212]:
# 保存df_posts
df_posts.to_csv('all_posts_new.csv', index=False, encoding='utf-8')

In [227]:
# 读取df_posts
df_posts = pd.read_csv('all_posts_new.csv',  encoding='utf-8', dtype={'user_id':str})
print(df_posts.shape)

(15907, 19)


In [228]:
# 获取cluster为 border_united_texas_trump
df_border = df_posts[df_posts['cluster'] == 'border_united_texas_trump']
print(df_border.shape)
# 获取其中非高光数据
df_border = df_border[df_border['query'] != 'highlight']
print(df_border.shape)
# 获取其中weibo数据
df_border_weibo = df_border[df_border['from'] == 'weibo']
print(df_border_weibo.shape)
# 获取其中1/3
df_border_weibo_remain = df_border_weibo.sample(frac=1/4)
# 把其它的数据cluster改为空
df_border_weibo = df_border_weibo.drop(df_border_weibo_remain.index)
df_border_weibo['cluster'] = ""
# 更新df_posts
df_posts.update(df_border_weibo)
# 查验
# 打印微博
df_border_weibo = df_posts[df_posts['cluster'] == 'border_united_texas_trump']
df_border_weibo = df_border_weibo[df_border_weibo['from'] == 'weibo']
print(df_border_weibo.shape)

print(df_posts[df_posts['cluster'] == 'border_united_texas_trump'].shape)



(2514, 19)
(2496, 19)
(1705, 19)
(431, 19)
(1235, 19)


In [229]:
# 查验高光数据
df_highlight = df_posts[df_posts['query'] == 'highlight']
# 按照cluster统计每组的数量
get_post_gorup_count(df_highlight)

,cluster,count
0,240_china_nuclear_pollution,6
1,70_billion_japan_water,5
2,Great_Wave_Kanagawa,12
3,border_united_texas_trump,18
4,cooling_water_nuclear_wastewater,9
5,japan_dead_fish,3
6,maga_win_trump_king,3
7,south_korea_nuclear_discharge,11
8,sue_TEPCO_japan,6
9,trump_more_votes_win,10


In [230]:
# 保存
df_posts.to_csv('all_posts_new.csv', index=False, encoding='utf-8')

In [231]:
df_posts = pd.read_csv('all_posts_new.csv',  encoding='utf-8', dtype={'user_id':str})

In [232]:
# 查看cluster为border_united_texas_trump的数据
df_border = df_posts[df_posts['cluster'] == 'border_united_texas_trump']
print(df_border.shape)

(1235, 19)


In [233]:
# 查看微博
df_border_weibo = df_border[df_border['from'] == 'weibo']
print(df_border_weibo.shape)

(431, 19)


In [234]:
# 按照from分组
print(df_border.groupby('from').size().reset_index(name='count'))

       from  count
0  facebook     28
1   twitter    776
2     weibo    431


In [235]:
# 获取2023.12.21 -> 2024.03.25之的数据
df = get_post_date_range(df_posts, '2023-08-10', '2024-03-25')
df.shape

(15173, 19)

In [236]:
# 按照
cluster_dict = {
    1:('Great_Wave_Kanagawa', "2021-04-20", "2021-04-29"),
    2:('foreign_affairs_questions', '2021-04-20', '2021-04-29'),
    3:('japan_nuclear_wastewater', '2021-04-20', '2021-04-29'),
    4:('radioactive_condemn_water', '2021-04-20', '2021-04-29'),
    5:('240_china_nuclear_pollution', '2023-08-21', '2023-08-30'),
    6:('70_billion_japan_water', '2023-08-21', '2023-08-30'),
    7:('cooling_water_nuclear_wastewater', '2023-08-21', '2023-08-30'),
    8:('south_korea_nuclear_discharge', '2023-08-21', '2023-09-01'),
    9:('sue_TEPCO_japan', '2023-08-21', '2023-08-30'),
    10:('radioactive_pollution_japan_sea', '2023-08-21', '2023-08-30'),
    11:('treatment_japan_waste_nuclear', '2023-08-21', '2023-08-30'),
    12: ('japan_dead_fish', '2023-12-01', '2023-12-10'),
    13: ('border_...', '2023-12-21', '2024-03-25'),
    14: ('maga_win_trump_king', '2023-12-21', '2024-03-25'),
    15: ('trump_primary_ballot', '2023-12-21', '2024-03-25'),
    16: ('trump_more_votes_win','2023-12-21', '2024-03-25'),
    17: ('republican_primary', '2023-12-21', '2024-03-25'),
    18: ('democratic_primary', '2023-12-21', '2024-03-25'),
    19 : ('desantis_quit', '2023-12-21', '2024-03-25'),
    20 : ('trump_plead_not_guilty', '2023-08-01', '2023-08-10')
}

In [244]:
cluster_election = list(cluster_dict.values())[13:]
cluster_election = [i[0] for i in cluster_election]
df_election = df_posts[df_posts['cluster'].isin(cluster_election)]
df_election.shape
print(df_election.groupby('from').size().reset_index(name='count'))

       from  count
0  facebook    172
1   twitter    783
2     weibo   3875


In [238]:
list(cluster_dict.values())[12:20]

[('border_...', '2023-12-21', '2024-03-25'),
 ('maga_win_trump_king', '2023-12-21', '2024-03-25'),
 ('trump_primary_ballot', '2023-12-21', '2024-03-25'),
 ('trump_more_votes_win', '2023-12-21', '2024-03-25'),
 ('republican_primary', '2023-12-21', '2024-03-25'),
 ('democratic_primary', '2023-12-21', '2024-03-25'),
 ('desantis_quit', '2023-12-21', '2024-03-25'),
 ('trump_plead_not_guilty', '2023-08-01', '2023-08-10')]